### Cross-Validation(교차검증)
* 과적합(Overfitting)을 방지해서 예측도를 높히기 위한 방법들
* 1. K-Fold
* 2. Stratified K-Fold
* 3. cross_val_score()
* 4. GridSearchCV

#### 1.K-Fold
* 모든 데이터가 최소 한번은 검증 데이터셋으로 사용되도록 한다.
* 1) Fold 세트 설정
* 2) for 루프로 학습/검증 데이터 추출 및 학습
* 3) Fold 세트별로 예측 성능 평균을 확인하기

In [8]:
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
import numpy as np

iris = load_iris()
print(iris.keys())
features = iris.data # = ['data'] , 독립변수
label = iris.target # 종속변수

print('붓꽃 데이터 세트 크기 ', features.shape, label.shape)
model = DecisionTreeClassifier(random_state=156)
model

# 5개의 폴드세트로 분리하는 KFold 객체를 생성
kfold = KFold(n_splits=5)
print(kfold)

for train_idx, test_idx in kfold.split(features):
    print('train \n', train_idx)
    print('test \n', test_idx)

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])
붓꽃 데이터 세트 크기  (150, 4) (150,)
KFold(n_splits=5, random_state=None, shuffle=False)
train 
 [ 30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47
  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]
test 
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
train 
 [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  

In [15]:
# 폴드 세트별 정확도 점수를 저장할 리스트 객체 선언
cv_accuracy = []
n_iter = 0

# KFold의 split() 함수를 호출하면 폴드명 학습용, 검증용 데이터의 row index를 array로 반환
for train_idx, test_idx in kfold.split(features):
    # 훈련데이터와 검증데이터 추출
    X_train, X_test = features[train_idx], features[test_idx]
    y_train, y_test = label[train_idx], label[test_idx]
    # 학습 및 예측
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    n_iter += 1
    
    # 예측점수 확인을 위해서 accuracy_score() 함수 사용
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print(f'#{n_iter} 교차 검증 정확도 : {accuracy} | 학습데이터크기 : {train_size} | 검증데이터 크기 : {test_size}')
    print(f'#{n_iter} 검증 세트 인덱스 : {test_idx} \n')
    cv_accuracy.append(accuracy)
    
print('## 교차 검증별 정확도 ', np.round(cv_accuracy, 4))
print('## 교차 검증별 정확도 평균 ', np.mean(cv_accuracy))

#1 교차 검증 정확도 : 1.0 | 학습데이터크기 : 120 | 검증데이터 크기 : 30
#1 검증 세트 인덱스 : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29] 

#2 교차 검증 정확도 : 0.9667 | 학습데이터크기 : 120 | 검증데이터 크기 : 30
#2 검증 세트 인덱스 : [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59] 

#3 교차 검증 정확도 : 0.8667 | 학습데이터크기 : 120 | 검증데이터 크기 : 30
#3 검증 세트 인덱스 : [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89] 

#4 교차 검증 정확도 : 0.9333 | 학습데이터크기 : 120 | 검증데이터 크기 : 30
#4 검증 세트 인덱스 : [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119] 

#5 교차 검증 정확도 : 0.7333 | 학습데이터크기 : 120 | 검증데이터 크기 : 30
#5 검증 세트 인덱스 : [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149] 

## 교차 검증별 정확도  [1.     0.9667 0.8667 0.9333 0.7333]
## 교차 검증별 정확도 평균  0.9


#### 2. Stratified K-Fold
* 학습데이터와 검증데이터 세트가 가지는 레이블(종속변수)의 분포도가 유사하도록 검증데이터를 추출하는 방법
* 일반 K-Fold와 다르게 레이블(종속변수)값을 넣어서 균형적으로 분할되도록 만듭니다.

In [21]:
import pandas as pd

iris = load_iris()

iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['label'] = iris.target
print(iris_df.label.value_counts())
print(iris_df['label'].iloc[0])
iris_df.head(3)

2    50
1    50
0    50
Name: label, dtype: int64
0


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0


In [23]:
# 일반 K-Fold 방식을 사용해서 레이블 분포를 확인
kfold = KFold(n_splits=5)
n_iter = 0
for train_idx, test_idx in kfold.split(iris_df):
    print('train ', train_idx)
    print('test ', test_idx)
    n_iter += 1
    label_train = iris_df['label'].iloc[train_idx]
    label_test = iris_df['label'].iloc[test_idx]
    print(f'#{n_iter} 학습 데이터 레이블 분포 \n {label_train.value_counts()}')
    print(f'#{n_iter} 검증 데이터 레이블 분포 \n {label_test.value_counts()}')

train  [ 30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47
  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119
 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]
test  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
#1 학습 데이터 레이블 분포 
 2    50
1    50
0    20
Name: label, dtype: int64
#1 검증 데이터 레이블 분포 
 0    30
Name: label, dtype: int64
train  [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  60  61  62  63  64  65
  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83
  84  85  86  87  88  89  90  91  92  93  94  95  96  97  

In [24]:
# StratifiedKFold 방식을 사용해서 레이블 분포를 확인하기
from sklearn.model_selection import StratifiedKFold

skfold = StratifiedKFold(n_splits=5)
n_iter = 0

for train_idx, test_idx in skfold.split(iris_df, iris_df.label):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_idx]
    label_test = iris_df['label'].iloc[test_idx]
    print(f'#{n_iter} 학습 데이터 레이블 분포 \n {label_train.value_counts()}')
    print(f'#{n_iter} 검증 데이터 레이블 분포 \n {label_test.value_counts()}')

#1 학습 데이터 레이블 분포 
 2    40
1    40
0    40
Name: label, dtype: int64
#1 검증 데이터 레이블 분포 
 2    10
1    10
0    10
Name: label, dtype: int64
#2 학습 데이터 레이블 분포 
 2    40
1    40
0    40
Name: label, dtype: int64
#2 검증 데이터 레이블 분포 
 2    10
1    10
0    10
Name: label, dtype: int64
#3 학습 데이터 레이블 분포 
 2    40
1    40
0    40
Name: label, dtype: int64
#3 검증 데이터 레이블 분포 
 2    10
1    10
0    10
Name: label, dtype: int64
#4 학습 데이터 레이블 분포 
 2    40
1    40
0    40
Name: label, dtype: int64
#4 검증 데이터 레이블 분포 
 2    10
1    10
0    10
Name: label, dtype: int64
#5 학습 데이터 레이블 분포 
 2    40
1    40
0    40
Name: label, dtype: int64
#5 검증 데이터 레이블 분포 
 2    10
1    10
0    10
Name: label, dtype: int64


In [38]:
iris = load_iris()
features = iris.data
label = iris.target

model = DecisionTreeClassifier(random_state=156)
skfold = StratifiedKFold(n_splits=5)

# 폴드 세트별 정확도 점수를 저장할 리스트 객체 선언
cv_accuracy = []
n_iter = 0

# KFold의 split() 함수를 호출하면 폴드명 학습용, 검증용 데이터의 row index를 array로 반환
for train_idx, test_idx in skfold.split(features, label):
    # 훈련데이터와 검증데이터 추출
    X_train, X_test = features[train_idx], features[test_idx]
    y_train, y_test = label[train_idx], label[test_idx]
    
    # 학습 및 예측
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    n_iter += 1
    
    # 예측점수 확인을 위해서 accuracy_score() 함수 사용
    accuracy = np.round(accuracy_score(y_test, pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    
    print(f'#{n_iter} 교차 검증 정확도 : {accuracy} | 학습데이터크기 : {train_size} | 검증데이터 크기 : {test_size}')
    print(f'#{n_iter} 검증 세트 인덱스 : {test_idx} \n')
    cv_accuracy.append(accuracy)
    
print('## 교차 검증별 정확도 ', np.round(cv_accuracy, 4))
print('## 교차 검증별 정확도 평균 {: .5f}'.format(np.mean(cv_accuracy)))

#1 교차 검증 정확도 : 0.9667 | 학습데이터크기 : 120 | 검증데이터 크기 : 30
#1 검증 세트 인덱스 : [  0   1   2   3   4   5   6   7   8   9  50  51  52  53  54  55  56  57
  58  59 100 101 102 103 104 105 106 107 108 109] 

#2 교차 검증 정확도 : 0.9667 | 학습데이터크기 : 120 | 검증데이터 크기 : 30
#2 검증 세트 인덱스 : [ 10  11  12  13  14  15  16  17  18  19  60  61  62  63  64  65  66  67
  68  69 110 111 112 113 114 115 116 117 118 119] 

#3 교차 검증 정확도 : 0.9 | 학습데이터크기 : 120 | 검증데이터 크기 : 30
#3 검증 세트 인덱스 : [ 20  21  22  23  24  25  26  27  28  29  70  71  72  73  74  75  76  77
  78  79 120 121 122 123 124 125 126 127 128 129] 

#4 교차 검증 정확도 : 0.9667 | 학습데이터크기 : 120 | 검증데이터 크기 : 30
#4 검증 세트 인덱스 : [ 30  31  32  33  34  35  36  37  38  39  80  81  82  83  84  85  86  87
  88  89 130 131 132 133 134 135 136 137 138 139] 

#5 교차 검증 정확도 : 1.0 | 학습데이터크기 : 120 | 검증데이터 크기 : 30
#5 검증 세트 인덱스 : [ 40  41  42  43  44  45  46  47  48  49  90  91  92  93  94  95  96  97
  98  99 140 141 142 143 144 145 146 147 148 149] 

## 교차 검증별 정확도  [0.9667 0.9667 0.9   

### 3. cross_val_score()
* cross_val_score는 K-Fold 사용할 때 수행했던 3단계의 교차검증을 보다 간편하게 해주는 방법
* cross_val_score() 함수는 폴드세트, 학습 및 예측, 평가를 한번에 수행해 준다.

In [40]:
from sklearn.model_selection import cross_val_score
iris = load_iris()
features = iris.data
label = iris.target

model = DecisionTreeClassifier(random_state=156)

scores = cross_val_score(model, features, label, scoring='accuracy', cv=5)
print(f'세트별 교차 검증 정확도 {np.round(scores,4)}')
print(f'교차 검증 정확도의 평균{np.round(np.mean(scores),4)}')



세트별 교차 검증 정확도 [0.9667 0.9667 0.9    0.9667 1.    ]
교차 검증 정확도의 평균0.96


#### 4. GridSearchCV
* Model을 생성할 때 인자로 주어지는 파라미터를 Hyper Parameter 라고 한다.
* 예측도 높이기위해서 Hyper Parameter Tunning이 필요
* GridSearchCV는 parameter를 여러개 다르게 설정한 모델을 만들어서 성능의 비교를 통해서 최적의 파라미터를 찾아주고, 
  교차검증도 해준다.

In [44]:
from sklearn.model_selection import GridSearchCV, train_test_split

iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=121)

model = DecisionTreeClassifier()

# max_depth : 트리의 최대 깊이
# min_samples_split : 노드를 분할하기 위한 최소한의 샘플데이터 수
param_dict = {'max_depth':[1,2,3], 'min_samples_split':[2,3]}

# 하이퍼 파라미터들을 5개의 학습, 검증 폴드로 나누어서 수행
# refit=True가 default 이고, True이면, 가장 좋은 파라미터로 설정하여 재 학습시킨다.
grid_tree = GridSearchCV(model, param_grid=param_dict, cv=5, refit=True)
print(type(grid_tree))

# GridSearchCV 객체를 fit() 으로 학습하기
grid_tree.fit(X_train, y_train)

#print(grid_tree.cv_results_)
grid_tree_df = pd.DataFrame(grid_tree.cv_results_)
grid_tree_df

<class 'sklearn.model_selection._search.GridSearchCV'>


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000799,0.000399,0.000200,0.000399,1,2,"{'max_depth': 1, 'min_samples_split': 2}",0.666667,0.708333,0.708333,0.708333,0.708333,0.700000,0.016667,5
1,0.000399,0.000488,0.000399,0.000489,1,3,"{'max_depth': 1, 'min_samples_split': 3}",0.666667,0.708333,0.708333,0.708333,0.708333,0.700000,0.016667,5
2,0.000600,0.000490,0.000199,0.000399,2,2,"{'max_depth': 2, 'min_samples_split': 2}",0.875000,1.000000,1.000000,1.000000,0.916667,0.958333,0.052705,3
3,0.000590,0.000483,0.000211,0.000421,2,3,"{'max_depth': 2, 'min_samples_split': 3}",0.875000,1.000000,1.000000,1.000000,0.916667,0.958333,0.052705,3
4,0.000000,0.000000,0.000591,0.000483,3,2,"{'max_depth': 3, 'min_samples_split': 2}",0.958333,1.000000,1.000000,1.000000,0.916667,0.975000,0.033333,1
5,0.000797,0.000398,0.000398,0.000488,3,3,"{'max_depth': 3, 'min_samples_split': 3}",0.958333,1.000000,1.000000,1.000000,0.916667,0.975000,0.033333,1


In [48]:
print(f'GridSearchCV 최적의 파라미터는 {grid_tree.best_params_}')
print(f'GridSearchCV 최고 예측 점수는 {grid_tree.best_score_}')

# GridSearchCV의 refit으로 재확습된 model 객체를 반환해준다.
best_model = grid_tree.best_estimator_

pred = best_model.predict(X_test)
print(f'검증 데이터 세트 정확도 {accuracy_score(y_test, pred)}')

GridSearchCV 최적의 파라미터는 {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 예측 점수는 0.975
검증 데이터 세트 정확도 0.9666666666666667
